# Problema del Viajante / Travelling Salesman Problem (TSP)

Vamos a tratar de solucionar el problema del viajante. En él, saliendo desde una ciudad tenemos que encontrar el camino óptimo pasando por cada ciudad una única vez y regresando al punto de partida. Cada camino tiene un coste así que habrá que buscar el que tenga menor coste.

El problema se modela usando **grafos ponderados completos no dirigidos** (las ciudades son los vértices).

Vamos a usar este grafo de ejemplo, el cual es muy simple:
<p align="center">
  <img src="imgs/tsp_example.png">
</p>

In [1]:
from math import inf

# Grafo de ejemplo, representado primero como lista de vértices y matriz de adyacencias
vertices = ["1","2","3","4"]
matriz = [[inf, 10, 15, 20],
          [10, inf, 35, 25],
          [15, 35, inf, 30],
          [20, 25, 30, inf]]

El primer algoritmo que vamos a implementar es el *ingenuo* o de *fuerza bruta*, ya que básicamente consiste en calcular todos los posibles caminos y quedarnos con el mejor

In [2]:
from itertools import permutations

'''
 Algoritmo de fuerza bruta sobre un grafo para encontrar
 la solución óptima al problema del viajante
 
 Entrada:
     matriz: list    -> matriz de adyacencias del grafo
     vertices: list  -> lista de vértices del grafo
     comienzo: str   -> nodo desde el que comenzamos el ciclo
 Salida:
     (list, float)   -> tupla formada por la lista de enlaces del 
                        camino encontrado y su longitud total
'''
def algoritmo_ingenuo_viajante(matriz: list, vertices: list, comienzo: str) -> (list, float):
    # Inicializamos los valores de retorno
    camino_optimo, coste_minimo = [], inf
    
    # Generamos los posibles caminos
    copia_vertices = vertices.copy()
    copia_vertices.remove(comienzo)
    posiciones = list(map(lambda vertice: int(vertice), copia_vertices))
    primera_posicion = int(comienzo) - 1
    posibles_caminos = permutations(posiciones)
    
    # Iteramos sobre cada posible camino
    for camino in posibles_caminos:
        coste_camino_actual = 0
 
        # Calculamos el coste del camino actual
        vertice_actual = primera_posicion
        for vertice in camino:
            coste_camino_actual += matriz[vertice_actual][vertice-1]
            vertice_actual = vertice - 1
        coste_camino_actual += matriz[vertice_actual][primera_posicion]
 
        # Actualizamos el mínimo
        if coste_camino_actual < coste_minimo:
            camino_optimo = list(camino)
            coste_minimo = coste_camino_actual

    # Terminamos de construir el camino óptimo como un ciclo
    camino_optimo = [primera_posicion + 1] + camino_optimo + [primera_posicion + 1]
    
    return camino_optimo, coste_minimo

In [3]:
algoritmo_ingenuo_viajante(matriz, vertices, vertices[0])

([1, 2, 4, 3, 1], 80)

In [4]:
# Veamos los distintos caminos óptimos comenzando por cada vértice. Deberían coincidir
for vertice in vertices:
    show(algoritmo_ingenuo_viajante(matriz, vertices, vertice))

([1, 2, 4, 3, 1], 80)

([2, 1, 3, 4, 2], 80)

([3, 1, 2, 4, 3], 80)

([4, 2, 1, 3, 4], 80)

Este algoritmo encuentra la solución óptima pero como es evidente, solo se puede usar para problemas con pocas ciudades. Esto se debe a su complejidad **factorial** (para N ciudades, es $O((N-1)!)$). Por ejemplo: para 10 ciudades hay (10-1)!/2=181440 rutas diferentes

Ahora vamos a implementar un algoritmo voraz para solucionar el problema del viajante. Escogemos el **Algoritmo del MST**. Básicamente consiste en construir primero el MST del grafo y después generar el tour óptimo del viajante recorriendo los vértices en el orden que indique un *recorrido preorden* del MST (cerrando el ciclo al final, obviamente)

Para encontrar el árbol recubridor mínimo vamos a reutilizar uno de los algoritmos ya implementados, el de **Prim**. Anteriormente comenzamos en el primer vértice de modo arbitrario, pero como el problema del viajante comienza en un nodo determinado, vamos a añadir ese parámetro al algoritmo.

Este sería su pseudocódigo:

<p align="center">
  <img src="imgs/psc_mst_tsp_tfg.png">
</p>

La manera de representar el grafo va a ser basada en la clase ***adjgraph*** mencionada en el notebook sobre grafos.

La clase ***adjgraph*** contiene:
* Una lista de vértices, cada uno representado por su etiqueta. Asumimos que son números enteros y ordenados del 1 en adelante.
* Una matriz de adyacencias (i.e., una lista de listas) cuadrada y simétrica que contiene en su casilla [i,j] la longitud del enlace que conecta dichos vértices, o infinito (*math.inf*) en caso de no estar conectados

In [5]:
'''
    Clase que representa un grafo ponderado NO DIRIGIDO 
    de n vértices como:
    - Una lista con las etiquetas de los vértices (ordenados
      de 1 a n)
    - Una matriz de adyacencias simétrica con la longitud de cada
      enlace e infinito en los enlaces que no existan
    Se implementan luego algunos métodos útiles
'''

class adjgraph(object):
    
    def __init__(self, vertices=[], matriz=[]):
        '''
            Inicializa un objeto adjgraph.
            Si no se pasan la matriz o los vértices,
            se usarán unos vacíos.
        '''
        self.__vertices = vertices
        self.__matriz = matriz

    def vertices(self):
        ''' Devuelve los vértices del grafo '''
        return self.__vertices
    
    def matriz(self):
        ''' Devuelve la matriz de adyacencias del grafo '''
        return self.__matriz

    def enlaces(self, vertice=None):
        ''' 
            Devuelve los enlaces del grafo. 
            Si se pasa un vértice como argumento,
            se devuelven los enlaces que contienen
            a dicho vértice. Si no, se devuelven
            todos los enlaces del grafo
        '''
        return self.__generar_enlaces(vertice)

    def anadir_vertice(self, vertice):
        ''' 
            Si el vértice "vertice" no está en 
            self.__vertices, se añade a la lista al
            final (debe ser el vértice n+1 tal y como
            hemos definido los vértices en este grafo)
            se añade en la matriz de enlaces con inf,
            y se devuelve True
            En otro caso, devuelve False. 
        '''
        if vertice not in self.__vertices:
            num_v = len(self.__vertices)
            if int(vertice) == num_v+1:
                self.__vertices.append(vertice)
                for lista in self.__matriz:
                    lista.append(inf)
                self.__matriz.append([inf]*(num_v+1))
                return True
        return False

    def anadir_enlace(self, vertice1, vertice2, longitud):
        ''' 
            Aquí como el grafo es no digirido, el orden de 
            los vértices solo es relevante a la hora de
            insertar los nuevos vértices en orden.
            Se crea un enlace entre los vértices de longitud 
            "longitud", añadiendo dichos vértices si es
            necesario y posible.
        '''
        indice1, indice2 = int(vertice1), int(vertice2)
        if vertice1 in self.__vertices and vertice2 in self.__vertices:
            i, j = indice1-1, indice2-1 
            if self.__matriz[i][j] == inf:
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
        elif vertice1 in self.__vertices:
            if self.anadir_vertice(vertice2):
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
        else:
            if abs(indice1-indice2) == 1:
                if self.anadir_vertice(vertice1):
                    self.anadir_vertice(vertice2)
                elif self.anadir_vertice(vertice2):
                    self.anadir_vertice(vertice1)
                else:
                    return
                i, j = indice1-1, indice2-1 
                self.__matriz[i][j] = longitud
                self.__matriz[j][i] = longitud
                    

    def __generar_enlaces(self, vertice=None):
        ''' 
            Un método estático que genera los enlaces del
            grafo. Si se pasa un vértice como argumento,
            se generan los enlaces que contienen
            a dicho vértice. Si no, se generan todos.
            Los enlaces son representados como 
            diccionarios con "e" (enlace) y "l" (longitud)
        '''
        enlaces = []
        n = len(self.__vertices)
        if vertice is None:
            for i in range(n):
                for j in range(i, n):
                    enlace = self.__crear_enlace(i, j)
                    if enlace is not None:
                        enlaces.append(enlace)
                    
        else:
            j = int(vertice - 1)
            for i in range(n):
                enlace = self.__crear_enlace(i, j)
                if enlace is not None:
                    enlaces.append(enlace)
                        
        return enlaces
    
    def __crear_enlace(self, fila, columna):
        ''' 
            Un método estático que crea un enlace
            accediendo a la matriz de adyacencias
            por una fila y columna determinadas
        '''
        if self.__matriz[fila][columna] != inf:
            return {"e" : {fila + 1, columna + 1},
                    "l" : self.__matriz[fila][columna]}
        else:
            return None

    def __str__(self):
        res = "vértices: "
        for k in self.__vertices:
            res += k + " "
        res += "\nenlaces: "
        for enlace in self.__generar_enlaces():
            res += str(enlace) + " "
        return res

In [6]:
'''
 Algoritmo de Prim para encontrar el árbol recubridor mínimo
 en un grafo conexo no dirigido, con una raíz determinada
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el árbol recubridor
                        mínimo
     comienzo: str   -> nodo desde el que comenzamos el MST
 Salida:
     (list, float)   -> tupla formada por la lista de enlaces del 
                        árbol recubridor mínimo y su longitud total
'''
def prim(grafo: adjgraph, comienzo: str) -> (set, float):
    longitud_total = 0                 # Longitud total del MST
    T = []                             # Lista de enlaces del MST
    primero = int(comienzo)            # Vértice en el que comenzamos
    primera_posicion = primero - 1     # Posición del vértice donde comenzamos
    n = len(grafo.vertices())          # Número de vértices del grafo
    matriz = grafo.matriz()            # Matriz de adyacencias del grafo
        
    # Inicializamos nuestros arrays según el vértice por el que empezamos
    mas_cercano = [primero] * n                 
    dist_min = []
    for i in range(n):
        if i == primera_posicion:
            dist_min.append(-1)
        else:
            dist_min.append(matriz[i][primera_posicion])

    for _ in range(n-1):
        min = inf
        for j in range(n):
            if j == primera_posicion:
                continue
            if 0 <= dist_min[j] and dist_min[j] < min:
                min = dist_min[j]
                k = j
        T.append({ "e" : {mas_cercano[k], k + 1},
                   "l" : min })
        longitud_total += min
        dist_min[k] = -1
        for j in range(n):
            if j == primera_posicion:
                continue
            if matriz[j][k] < dist_min[j]:
                dist_min[j] = matriz[j][k]
                mas_cercano[j] = k + 1
    
    return T, longitud_total

In [7]:
'''
 Función que devuelve una lista con los vértices del
 árbol recibido, recorriéndolo en preorden
 
 Entrada:
     arbol: list -> lista de enlaces del arbol

 Salida:
     list        -> lista con los vértices del recorrido preorden
'''
def vertices_preorden(arbol: list) -> list:    
    vertices = []       # Lista de vertices del recorrido preorden
    
    for enlace in arbol:
        for vertice in enlace["e"]:
            if vertice not in vertices:
                vertices.append(vertice)
    
    return vertices

In [8]:
'''
 Función que devuelve los enlaces del ciclo que recorre
 los vértices recibidos en ese mismo orden, y la longitud
 total de dicho ciclo
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el ciclo óptimo
     vertices: list  -> lista de vertices en determinado orden

 Salida:
     (list, float)   -> tupla formada por la lista de enlaces del 
                        ciclo y su longitud total
'''
def enlaces_tour_viajante(grafo: adjgraph, vertices: list) -> (list, float):
    longitud_total = 0             # Longitud total del ciclo
    ciclo = []                     # Lista de enlaces del ciclo
    matriz = grafo.matriz()        # Matriz de adyacencias del grafo
    n = len(vertices)              # Número de vértices
    for pos in range(n-1):
        vertice1, vertice2 = vertices[pos], vertices[pos + 1]
        longitud = matriz[vertice1 - 1][vertice2 - 1]
        ciclo.append({ "e" : {vertice1, vertice2},
                       "l" : longitud })
        longitud_total += longitud
        
    # Cerramos el ciclo
    longitud = matriz[vertice2 - 1][vertices[0] - 1]
    ciclo.append({ "e": {vertice2, vertices[0]},
                   "l": longitud })
    longitud_total += longitud
    
    return ciclo, longitud_total

In [9]:
'''
 Algoritmo del MST para encontrar una buena aproximación a la
 solución del problema del viajante en un grafo completo no dirigido
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el ciclo óptimo
     comienzo: str   -> nodo desde el que comenzamos el MST

 Salida:
     (list, float)   -> tupla formada por la lista de enlaces del 
                        camino encontrado y su longitud total
'''
def mst_viajante(grafo: adjgraph, comienzo: str) -> (set, float):
    mst, _ = prim(grafo, comienzo)
    
    vertices = vertices_preorden(mst)
    
    H, longitud_total = enlaces_tour_viajante(grafo, vertices)
    
    return H, longitud_total

Probamos el algoritmo con el ejemplo anterior, comenzando arbitrariamente en el primer vértice

In [10]:
grafo_tsp_adj = adjgraph(vertices, matriz)

mst_viajante(grafo_tsp_adj, vertices[0])

([{'e': {1, 2}, 'l': 10},
  {'e': {2, 3}, 'l': 35},
  {'e': {3, 4}, 'l': 30},
  {'e': {1, 4}, 'l': 20}],
 95)

Este algoritmo nos proporciona una **aproximación** a la solución óptima, ya que da una solución que no solo depende del grafo en cuestión, sino también del vértice en el que comencemos. Vamos a comprobarlo con el ejemplo anterior:

In [11]:
for vertice in vertices:
    show(mst_viajante(grafo_tsp_adj, vertice))

([{'e': {1, 2}, 'l': 10},
  {'e': {2, 3}, 'l': 35},
  {'e': {3, 4}, 'l': 30},
  {'e': {1, 4}, 'l': 20}],
 95)

([{'e': {1, 2}, 'l': 10},
  {'e': {2, 3}, 'l': 35},
  {'e': {3, 4}, 'l': 30},
  {'e': {1, 4}, 'l': 20}],
 95)

([{'e': {1, 3}, 'l': 15},
  {'e': {2, 3}, 'l': 35},
  {'e': {2, 4}, 'l': 25},
  {'e': {1, 4}, 'l': 20}],
 95)

([{'e': {1, 4}, 'l': 20},
  {'e': {2, 4}, 'l': 25},
  {'e': {2, 3}, 'l': 35},
  {'e': {1, 3}, 'l': 15}],
 95)

### Como podemos comprobar, el algoritmo no es óptimo ya que nos devuelve ciclos con longitud mayor de la óptima que calculamos antes con el algoritmo ingenuo. De hecho, son EL PEOR camino.

Vamos a implementar ahora otro algoritmo voraz, el del **vecino más próximo** pero adaptado al TSP (cerrando el ciclo para simular el volver a la ciudad de origen). Este sería su pseudocódigo:

<p align="center">
  <img src="imgs/psc_nn_tsp_tfg.png">
</p>

In [12]:
import heapq as hq

'''
 Función que genera los enlaces del grafo que contienen
 a dicho "vertice" y no a los que estén en "filtro".
 
 Entrada:
     matriz: list -> matriz de adyacencias del grafo
     vertice: int -> vértice del que buscamos los enlaces
     filtro: set  -> conjunto de vértices que no queremos en
                     los enlaces

 Salida:
     list -> heap (como una lista) de enlaces 
             (como tuplas (longitud, enlace))
'''
def generar_enlaces_filtrados(matriz: list, vertice: int, filtro: set) -> list:
    distancias = matriz[vertice - 1]
    heap = []
    for pos in range(len(distancias)):
        vecino = pos + 1
        distancia = distancias[pos]
        if distancia != inf and vecino not in filtro:
            hq.heappush(heap, (distancia, {vertice, vecino}))

    return heap

In [13]:
'''
 Función que selecciona el enlace más corto,
 lo extrae del heap y lo devuelve
 
 Entrada:
     heap: list -> lista de enlaces con estructura de min heap
 Salida:
     dict -> enlace de la raíz del heap, en forma de diccionario
'''
def seleccionar_viajante(heap: list) -> dict:
    longitud, enlace = hq.heappop(heap)
    return { "e" : enlace, "l" : longitud }

In [14]:
'''
 Algoritmo del vecino más próximo para encontrar una buena aproximación a
 la solución del problema del viajante en un grafo completo no dirigido
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos el ciclo óptimo
     comienzo: str   -> nodo desde el que comenzamos el ciclo
 Salida:
     (list, float)   -> tupla formada por la lista de enlaces del 
                        camino encontrado y su longitud total
'''
def vecino_proximo_viajante(grafo: adjgraph, comienzo: str) -> (set, float):
    longitud_total = 0                 # Longitud total del camino
    T = []                             # Lista de enlaces del camino
    primero = int(comienzo)            # Vértice en el que comenzamos
    n = len(grafo.vertices())          # Número de vértices del grafo
    matriz = grafo.matriz()            # Matriz de adyacencias del grafo
    vertice_actual = primero           # Vértice en el que estamos
    visitados = {primero}              # Conjunto de vértices visitados

    # Bucle voraz
    for _ in range(n-1):
        enlaces = generar_enlaces_filtrados(matriz, vertice_actual, visitados)
        enlace_optimo = seleccionar_viajante(enlaces)
        T.append(enlace_optimo)
        longitud_total += enlace_optimo["l"]
        vecino_mas_proximo = (enlace_optimo["e"] - {vertice_actual}).pop()
        visitados.add(vecino_mas_proximo)
        vertice_actual = vecino_mas_proximo
    
    # Cerramos el ciclo
    longitud = matriz[vertice_actual - 1][primero - 1]
    T.append({ "e": {vertice_actual, primero},
               "l": longitud })
    longitud_total += longitud
    
    return T, longitud_total

Probamos el algoritmo con el ejemplo anterior, comenzando arbitrariamente en el primer vértice

In [15]:
vecino_proximo_viajante(grafo_tsp_adj, vertices[0])

([{'e': {1, 2}, 'l': 10},
  {'e': {2, 4}, 'l': 25},
  {'e': {3, 4}, 'l': 30},
  {'e': {1, 3}, 'l': 15}],
 80)

Hemos decidido usar **min heaps** para representar los enlaces generados, teniendo el más corto en la raíz. De este modo, el *seleccionar_viajante()* será la propia raíz. Esto permitirá agilizar este último método, aunque por otro lado haya que crear un heap a cada iteración del bucle voraz, según el vértice que estemos explorando.

Este algoritmo también nos proporciona una **aproximación** a la solución óptima, ya que da una solución que no solo depende del grafo en cuestión, sino también del vértice en el que comencemos. Vamos a comprobarlo con el ejemplo anterior:

In [16]:
for vertice in vertices:
    show(vecino_proximo_viajante(grafo_tsp_adj, vertice))

([{'e': {1, 2}, 'l': 10},
  {'e': {2, 4}, 'l': 25},
  {'e': {3, 4}, 'l': 30},
  {'e': {1, 3}, 'l': 15}],
 80)

([{'e': {1, 2}, 'l': 10},
  {'e': {1, 3}, 'l': 15},
  {'e': {3, 4}, 'l': 30},
  {'e': {2, 4}, 'l': 25}],
 80)

([{'e': {1, 3}, 'l': 15},
  {'e': {1, 2}, 'l': 10},
  {'e': {2, 4}, 'l': 25},
  {'e': {3, 4}, 'l': 30}],
 80)

([{'e': {1, 4}, 'l': 20},
  {'e': {1, 2}, 'l': 10},
  {'e': {2, 3}, 'l': 35},
  {'e': {3, 4}, 'l': 30}],
 95)

### Como podemos comprobar, el algoritmo no es óptimo ya que comenzando en el último vértice el algoritmo nos devuelve un ciclo con longitud mayor. De hecho, es EL PEOR camino.

Vamos ahora a probarlos en un grafo más grande, el de la memoria del TFG

<p align="center">
  <img src="imgs/grafo_ejemplo_tfg_tsp.png">
</p>

In [17]:
# Grafo de ejemplo del TFG, representado como adjgraph
vertices_tfg = ["1","2","3","4","5","6","7","8"]
matriz_tfg = [[inf, 4, 2, 4, 4, 2, 5, 5],
              [4, inf, 5, 4, 3, 3, 2, 3],
              [2, 5, inf, 4, 3, 3, 4, 4],
              [4, 4, 4, inf, 3, 5, 2, 3],
              [4, 3, 3, 3, inf, 3, 2, 4],
              [2, 3, 3, 5, 3, inf, 4, 3],
              [5, 2, 4, 2, 2, 4, inf, 3],
              [5, 3, 4, 3, 4, 3, 3, inf]]

In [18]:
algoritmo_ingenuo_viajante(matriz_tfg, vertices_tfg, vertices_tfg[0])

([1, 3, 5, 2, 7, 4, 8, 6, 1], 20)

In [19]:
# Este algoritmo, como es de esperar, nos da la misma solución (la óptima) empezando desde cualquier vértice

for vertice in vertices_tfg:
    show(algoritmo_ingenuo_viajante(matriz_tfg, vertices_tfg, vertice))

([1, 3, 5, 2, 7, 4, 8, 6, 1], 20)

([2, 5, 3, 1, 6, 8, 4, 7, 2], 20)

([3, 1, 6, 2, 8, 4, 7, 5, 3], 20)

([4, 5, 3, 1, 6, 8, 2, 7, 4], 20)

([5, 2, 7, 4, 8, 6, 1, 3, 5], 20)

([6, 1, 3, 5, 2, 7, 4, 8, 6], 20)

([7, 2, 5, 3, 1, 6, 8, 4, 7], 20)

([8, 2, 6, 1, 3, 5, 7, 4, 8], 20)

In [20]:
grafo_tsp_adj_tfg = adjgraph(vertices_tfg, matriz_tfg)

mst_viajante(grafo_tsp_adj_tfg, vertices_tfg[0])

([{'e': {1, 3}, 'l': 2},
  {'e': {3, 6}, 'l': 3},
  {'e': {2, 6}, 'l': 3},
  {'e': {2, 7}, 'l': 2},
  {'e': {4, 7}, 'l': 2},
  {'e': {4, 5}, 'l': 3},
  {'e': {5, 8}, 'l': 4},
  {'e': {1, 8}, 'l': 5}],
 24)

In [21]:
vecino_proximo_viajante(grafo_tsp_adj_tfg, vertices_tfg[0])

([{'e': {1, 3}, 'l': 2},
  {'e': {3, 5}, 'l': 3},
  {'e': {5, 7}, 'l': 2},
  {'e': {2, 7}, 'l': 2},
  {'e': {2, 6}, 'l': 3},
  {'e': {6, 8}, 'l': 3},
  {'e': {4, 8}, 'l': 3},
  {'e': {1, 4}, 'l': 4}],
 22)

Estos son algunos de los resultados mencionados en la memoria, pero veamos comenzando desde cada vértice, calculando el % de error sobre la solución óptima que ya conocemos:

In [22]:
coste_optimo = algoritmo_ingenuo_viajante(matriz_tfg, vertices_tfg, vertices_tfg[0])[-1]
errores_mst_viajante, errores_vecino_proximo_viajante = [],[]

for vertice in vertices_tfg:
    coste_mst_viajante = mst_viajante(grafo_tsp_adj_tfg, vertice)[-1]
    error_mst_viajante = float(100*(1 - coste_optimo / coste_mst_viajante))
    errores_mst_viajante.append(f"{error_mst_viajante:.2f}%")
    
    coste_vecino_proximo_viajante = vecino_proximo_viajante(grafo_tsp_adj_tfg, vertice)[-1]
    error_vecino_proximo_viajante = float(100*(1 - coste_optimo / coste_vecino_proximo_viajante))
    errores_vecino_proximo_viajante.append(f"{error_vecino_proximo_viajante:.2f}%")

In [23]:
show("Errores MST viajante", errores_mst_viajante)
show("Errores Vecino Más Próximo viajante", errores_vecino_proximo_viajante)

'Errores MST viajante' ['16.67%', '0.00%', '16.67%', '0.00%', '13.04%', '23.08%', '0.00%', '0.00%']

'Errores Vecino Más Próximo viajante' ['9.09%', '0.00%', '9.09%', '0.00%', '9.09%', '9.09%', '0.00%', '0.00%']

### Como podemos comprobar, en este ejemplo del TFG obtenemos la solución óptima en la mitad de los casos, y en los que no, aproximaciones con errores no peores de lo estimado teóricamente en la memoria. De todos modos, las pruebas deberían realizarse sobre grafos más grandes y aleatorios para ser más determinantes.

Para terminar el notebook, adicionalmente, vamos a implementar un algoritmo para comprobar si un grafo ponderado no dirigido cumple la desigualdad triangular, ya que como se explica en la memoria del TFG, los dos algoritmos voraces anteriores solo garantizan su nivel de aproximación cuando el grafo cumple esta propiedad.

## Algoritmo para comprobar que un grafo ponderado no dirigido cumple la desigualdad triangular

Básicamente el proceso va a ser:
- Generar todos los triángulos del grafo
- Comprobar que todos cumplen la desigualdad triangular

Vamos a representar los triángulos como diccionarios con:
- "vértices": conjunto con las etiquetas de sus 3 vértices
- "enlaces": lista con sus tres enlaces (con nuestro formato anterior de diccionario)


In [24]:
# Ejemplo de triángulo representado como diccionario
triangulo = {"vertices": {"1","2","3"},
              "enlaces": [{'e': {1, 2}, 'l': 10},
                          {'e': {2, 3}, 'l': 25},
                          {'e': {1, 3}, 'l': 30}]}

In [25]:
'''
 Función que recibe un triángulo y comprueba si
 sus enlaces cumplen la desigualdad triangular
 
 Entrada:
     triangulo: dict -> diccionario con los vértices
                        y enlaces del triángulo

 Salida:
     bool -> Si cumple la desigualdad triangular o no
'''
def triangulo_cumple_desigualdad_triangular(triangulo: dict) -> bool:
    if len(triangulo["vertices"]) != 3:
        print("Error: Triángulo incorrecto")
        return False
    
    enlaces = triangulo["enlaces"]
    d1, d2, d3 = enlaces[0]["l"], enlaces[1]["l"], enlaces[2]["l"]
    return (d1 <= d2 + d3 and d2 <= d1 + d3 and d3 <= d1 + d2)

In [26]:
''' 
 Función que crea un enlace accediendo a la matriz
 de adyacencias por una fila y columna determinadas
    
Entrada:
     matriz: list -> matriz de adyacencias del grafo
     fila: int    -> posición del primer vértice del enlace
     columna: int -> posición del segundo vértice del enlace

 Salida:
     dict -> enlace creado o None en caso de no existir
'''
def crear_enlace(matriz: list, fila: int, columna: int) -> dict:
    if matriz[fila][columna] != inf:
        return {"e" : {fila + 1, columna + 1},
                "l" : matriz[fila][columna]}
    else:
        return None

In [27]:
from itertools import combinations

'''
 Función que genera los triángulos del grafo.
 
 Entrada:
     grafo: adjgraph -> grafo donde buscamos
                        los triángulos

 Salida:
     set -> conjunto de triángulos del grafo
'''
def generar_triangulos_grafo(grafo: adjgraph) -> list:
    matriz = grafo.matriz()      # Matriz de adyacencias del grafo
    vertices = grafo.vertices()  # Vertices del grafo
    n = len(vertices)
    triangulos = []
    
    for vertice in vertices:
        copia_vertices = vertices.copy()
        copia_vertices.remove(vertice)
        pares = list(combinations(copia_vertices, 2))
        pos = int(vertice) - 1
        for par in pares:
            [pos_v1, pos_v2] = list(map(lambda v: int(v) - 1, par))
            verts = {vertice, par[0], par[1]}
            if any(map(lambda t: verts == t["vertices"], triangulos)):
                continue
            triangulo = []
            triangulo.append(crear_enlace(matriz, pos_v1, pos_v2))
            triangulo.append(crear_enlace(matriz, pos, pos_v1))
            triangulo.append(crear_enlace(matriz, pos, pos_v2))
            triangulos.append({"vertices": verts,
                               "enlaces": triangulo})

    return triangulos

In [28]:
'''
 Función que recibe un grafo y comprueba si
 cumple la desigualdad triangular
 
 Entrada:
     grafo: adjgraph -> grafo donde queremos comprobar
                        la desigualdad triangular

 Salida:
     bool -> Si cumple la desigualdad triangular o no
'''
def grafo_cumple_desigualdad_triangular(grafo: adjgraph) -> bool:
    triangulos = generar_triangulos_grafo(grafo)
    return all(map(triangulo_cumple_desigualdad_triangular, triangulos))

In [29]:
show(f"¿Cumple el primer grafo de ejemplo la desigualdad triangular? {grafo_cumple_desigualdad_triangular(grafo_tsp_adj)}")
show(f"¿Cumple el segundo grafo de ejemplo la desigualdad triangular? {grafo_cumple_desigualdad_triangular(grafo_tsp_adj_tfg)}")

'¿Cumple el primer grafo de ejemplo la desigualdad triangular? False'

'¿Cumple el segundo grafo de ejemplo la desigualdad triangular? True'

### Para el primer grafo era sencillo comprobar que no se cumplía la propiedad, pero en el segundo no es evidente a simple vista, por lo que este algoritmo resulta útil. De todos modos para grafos muy grandes y densos, puede no resultar eficiente.